<a href="https://colab.research.google.com/github/mzohaibnasir/NeuralNotes/blob/main/04_deepDiveIntoBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc

from torch.utils.data import default_collate

# from miniai.training import *

In [3]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [4]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path

from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close

In [5]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [6]:
import logging
logging.disable(logging.WARNING)


# Hugging Face Datasets

In [25]:
"""Use the load_dataset_builder() function to load a dataset builder and inspect a dataset’s attributes without committing to downloading it:"""
name = "fashion_mnist"
from datasets import load_dataset_builder
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)




/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for fashion_mnist contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fashion_mnist
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



In [21]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [24]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31304107, num_examples=60000, shard_lengths=None, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5235060, num_examples=10000, shard_lengths=None, dataset_name='fashion_mnist')}

In [30]:
dsd = load_dataset(name)
dsd # DatasetDict  -  dataset dictionary

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [31]:
train, test = dsd['train'], dsd['test']   # fetching element from DatasetDict using keys

train.shape, test.shape

((60000, 2), (10000, 2))

In [33]:
train[0], test[0]  # here we are getting dictionaries, not tuple

({'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  'label': 9},
 {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  'label': 9})

In [34]:
x , y = 'image' ,'label' # for
x , y

('image', 'label')

In [38]:
x, y = ds_builder.info.features
x,y

('image', 'label')

In [43]:
img = train[0][x]
img

In [44]:
train[0][y]

9

In [48]:
train[:5][x], train[:5][y]

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

In [49]:
featy = train.features[y]  # label names
featy

ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [51]:
xb,yb = train[:5][x] , train[:5][y]
xb,yb

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

In [52]:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [55]:
train['label'][:5]

[9, 0, 0, 3, 0]

In [56]:
# collate function will return dictionary


def collate_fn(b):
  return {
      x:torch.stack([
          TF.to_tensor(o[x]) for o in b
      ]),

      y:tensor([o[y] for o in b])
  }

In [57]:
dl = DataLoader(train, collate_fn=collate_fn, batch_size=16)
b = next(iter(dl))

b[x].shape, b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [58]:
def transforms(b):
  b[x] = [TF.to_tensor(o) for o in b[x]]
  return b